<a href="https://colab.research.google.com/github/yemanzhongting/MultiCity/blob/main/%E6%A8%A1%E6%80%81%E7%9F%A9%E9%98%B5%E6%90%AD%E5%BB%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt

import pickle as pkl
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot
# from keras.optimizers import Adam
# from keras.utils import plot_model
from keras.callbacks import EarlyStopping
import pydot as pyd
from keras.utils.vis_utils import plot_model, model_to_dot
keras.utils.vis_utils.pydot = pyd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import os
import sys
import math
from collections import Counter
import numpy as np
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import nltk

In [5]:
def load_data(in_file):
    cn = []
    en = []
    num_examples = 0
    with open(in_file, 'r') as f:
        for line in f:
            line = line.strip().split("\t")
            
            # en.append(["BOS"] + nltk.word_tokenize(line[0].lower()) + ["EOS"])
            en.append(nltk.word_tokenize(line[0].lower()))
            # split chinese sentence into characters
            # cn.append(["BOS"] + [c for c in line[1].split(' ')] + ["EOS"])
            cn.append([c for c in line[1].split(' ')] )
    return en, cn

In [6]:
import nltk
nltk.download('punkt')
train_file = "/content/drive/MyDrive/data/sv-sub.txt"
dev_file = "nmt/en-cn/dev.txt"
train_en, train_cn = load_data(train_file)
# dev_en, dev_cn = load_data(dev_file)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [7]:
type(train_en)

list

In [8]:
UNK_IDX = 0
PAD_IDX = 1
def build_dict(sentences, max_words=50000):
    word_count = Counter()
    for sentence in sentences:
        for s in sentence:
            word_count[s] += 1
    ls = word_count.most_common(max_words)
    total_words = len(ls) #+ 2
    # word_dict = {w[0]: index+2 for index, w in enumerate(ls)}
    word_dict = {w[0]: index+1 for index, w in enumerate(ls)}
    # word_dict["UNK"] = UNK_IDX
    # word_dict["PAD"] = PAD_IDX
    return word_dict, total_words

en_dict, en_total_words = build_dict(train_en)
cn_dict, cn_total_words = build_dict(train_cn)
inv_en_dict = {v: k for k, v in en_dict.items()}
inv_cn_dict = {v: k for k, v in cn_dict.items()}

In [11]:
from keras.utils import np_utils

def words_2_ints(words):
    ints = []
    for itmp in words:
        ints.append(en_dict.get(itmp, 0))
    return ints
def words_2_one_hot(words, num_classes):
    return np_utils.to_categorical(words_2_ints(words), num_classes=num_classes)
import tensorflow as tf
# from tf.keras.preprocessing.sequence import pad_sequences
train_en_pad=[]
for i in train_en:
  tmp=tf.keras.preprocessing.sequence.pad_sequences(words_2_one_hot(i,len(en_dict)+1).T, padding="post", maxlen=10,value=0).T
  train_en_pad.append(tmp)
train_en_pad_arr=np.array(train_en_pad)
train_en_pad_arr.shape

(20790, 10, 140)

In [14]:
from keras.utils import np_utils

def words_2_ints(words):
    ints = []
    for itmp in words:
        ints.append(cn_dict.get(itmp, 0))
    return ints
def words_2_one_hot(words, num_classes):
    return np_utils.to_categorical(words_2_ints(words), num_classes=num_classes)
import tensorflow as tf
# from tf.keras.preprocessing.sequence import pad_sequences
train_cn_pad=[]
for i in train_cn:
  tmp=tf.keras.preprocessing.sequence.pad_sequences(words_2_one_hot(i,len(cn_dict)+1).T, padding="post", maxlen=10,value=0).T
  train_cn_pad.append(tmp)
train_cn_pad_arr=np.array(train_cn_pad)
train_cn_pad_arr.shape

(20790, 10, 155)

In [15]:

# model=Sequential()
# model.add(LSTM(256,dropout_W=0.2,dropout_u=0.2,input_shape(seq_length,128)))
# model.add(Dropout(0.2))
# model.add(Dense(128,activation="sigmoid"))
# model,compile(loss="mse",optimizer="adm")
import random
import numpy as np
import matplotlib.pyplot as plt

import pickle as pkl
import keras
from keras.models import Sequential, Model, load_model
from keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input, BatchNormalization, \
    multiply, concatenate, Flatten, Activation, dot

from tensorflow.keras.optimizers import Adam # - Works

# from keras.optimizers import Adam
# from keras.utils import plot_model
from keras.callbacks import EarlyStopping
import pydot as pyd
# from keras.utils.vis_utils import plot_model, model_to_dot
keras.utils.vis_utils.pydot = pyd

In [28]:
n_hidden = 100
input_train = Input(shape=(train_en_pad_arr.shape[1], train_en_pad_arr.shape[2]))
output_train = Input(shape=(train_cn_pad_arr.shape[1], train_cn_pad_arr.shape[2]))

https://levelup.gitconnected.com/building-seq2seq-lstm-with-luong-attention-in-keras-for-time-series-forecasting-1ee00958decb

In [29]:
encoder_stack_h, encoder_last_h, encoder_last_c = LSTM(
    n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2, 
    return_state=True, return_sequences=True)(input_train)
print(encoder_stack_h)
print(encoder_last_h)
print(encoder_last_c)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 100), dtype=tf.float32, name=None), name='lstm_2/transpose_1:0', description="created by layer 'lstm_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.float32, name=None), name='lstm_2/while:4', description="created by layer 'lstm_2'")
KerasTensor(type_spec=TensorSpec(shape=(None, 100), dtype=tf.float32, name=None), name='lstm_2/while:5', description="created by layer 'lstm_2'")


In [30]:
encoder_last_h = BatchNormalization(momentum=0.6)(encoder_last_h)
encoder_last_c = BatchNormalization(momentum=0.6)(encoder_last_c)

In [31]:
decoder_input = RepeatVector(output_train.shape[1])(encoder_last_h)
print(decoder_input)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 100), dtype=tf.float32, name=None), name='repeat_vector_1/Tile:0', description="created by layer 'repeat_vector_1'")


In [32]:
decoder_stack_h = LSTM(n_hidden, activation='elu', dropout=0.2, recurrent_dropout=0.2,
 return_state=False, return_sequences=True)(
 decoder_input, initial_state=[encoder_last_h, encoder_last_c])
print(decoder_stack_h)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 100), dtype=tf.float32, name=None), name='lstm_3/transpose_1:0', description="created by layer 'lstm_3'")


In [33]:
attention = dot([decoder_stack_h, encoder_stack_h], axes=[2, 2])
attention = Activation('softmax')(attention)
print(attention)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 10), dtype=tf.float32, name=None), name='activation_1/Softmax:0', description="created by layer 'activation_1'")


In [34]:
context = dot([attention, encoder_stack_h], axes=[2,1])
context = BatchNormalization(momentum=0.6)(context)
print(context)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 100), dtype=tf.float32, name=None), name='batch_normalization_5/batchnorm/add_1:0', description="created by layer 'batch_normalization_5'")


In [35]:
decoder_combined_context = concatenate([context, decoder_stack_h])
print(decoder_combined_context)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 200), dtype=tf.float32, name=None), name='concatenate_1/concat:0', description="created by layer 'concatenate_1'")


In [36]:
out = TimeDistributed(Dense(output_train.shape[2]))(decoder_combined_context)
print(out)

KerasTensor(type_spec=TensorSpec(shape=(None, 10, 155), dtype=tf.float32, name=None), name='time_distributed_1/Reshape_1:0', description="created by layer 'time_distributed_1'")


In [37]:
model = Model(inputs=input_train, outputs=out)
opt = Adam(lr=0.01, clipnorm=1)
model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mae'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 10, 140)]    0                                            
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, 10, 100), (N 96400       input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_3 (BatchNor (None, 100)          400         lstm_2[0][1]                     
__________________________________________________________________________________________________
repeat_vector_1 (RepeatVector)  (None, 10, 100)      0           batch_normalization_3[0][0]      
____________________________________________________________________________________________

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


In [38]:
epc = 5
es = EarlyStopping(monitor='val_loss', mode='min', patience=50)
history = model.fit(train_en_pad_arr, train_cn_pad_arr, validation_split=0.2, 
                    epochs=epc, verbose=1, callbacks=[es], 
                    batch_size=100)
train_mae = history.history['mae']
valid_mae = history.history['val_mae']
 
model.save('model_forecasting_seq2seq.h5')

Epoch 1/5
167/167 [==============================] - 30s 131ms/step - loss: 0.0075 - mae: 0.0244 - val_loss: 0.0062 - val_mae: 0.0139
Epoch 2/5
167/167 [==============================] - 21s 128ms/step - loss: 0.0062 - mae: 0.0147 - val_loss: 0.0062 - val_mae: 0.0142
Epoch 3/5
167/167 [==============================] - 21s 125ms/step - loss: 0.0062 - mae: 0.0142 - val_loss: 0.0062 - val_mae: 0.0140
Epoch 4/5
167/167 [==============================] - 21s 128ms/step - loss: 0.0061 - mae: 0.0139 - val_loss: 0.0061 - val_mae: 0.0137
Epoch 5/5
167/167 [==============================] - 21s 126ms/step - loss: 0.0061 - mae: 0.0138 - val_loss: 0.0061 - val_mae: 0.0137


In [46]:
model.predict()

TypeError: ignored

In [44]:
train_en_pad_arr[0].shape

(10, 140)